# COMP90016 - Assignment 2
Version 1 Last edited 10/4/2024

## Semester 1, 2024

In [ ]:
NAME = "Keziah Tikno"

ID = "1319716"

This assignment should be completed by each student individually. Make sure you read this entire document, and ask for help if anything is not clear. Any changes or clarifications to this document will be announced via the LMS.

Please make sure you review the University's rules on academic integrity: https://academicintegrity.unimelb.edu.au/

You submission must be your own work. Do not copy material from other students, from the internet or from AI tools. 

Your completed notebook file containing all your answers will be turned in via Canvas. Please also submit an HTML file with the output cleared.

To complete the assignment, finish the tasks in this notebook.

The tasks are a combination of writing your own code, interpreting the results and answering related short-answer questions.

In some cases, we have provided test input and test output that you can use to try out your solutions. These tests are just samples and are **not** exhaustive - they may warn you if you've made a mistake, but they are not guaranteed to. It's up to you to decide whether your code is correct.

**Remember to save your work early and often.**

## Marking

Cells that must be completed to receive marks are clearly labelled. Some cells are code cells, in which you must complete the code to solve a problem. Others are markdown cells, in which you must write your answers to short-answer questions. 

Cells that must be completed to receive marks are labelled like this:

`# -- GRADED CELL (1 mark) - complete this cell --`

Some graded cells are code cells, in which you must complete the code to solve a problem. Other graded cells are markdown cells, in which you must write your answers to short-answer questions. 

You will see the following text in graded code cells:

```
# YOUR CODE HERE
raise NotImplementedError()
```

***You must remove the `raise NotImplementedError()` line from the cell, and replace it with your solution.***

Only add answers to graded cells. If you want to import a library or use a helper function, this must be included in a graded cell.

Only graded cells will be marked.
**Don't make changes outside graded cells, and don't add or remove cells from the notebook**.

>Word limits, where stated, will be strictly enforced. Answers exceeding the limit **will not be marked**.

>Run-time limits will be imposed for each coding question. The run-time of a code cell can be calculated by including `%time` at the top of your cell. Cells exceeding the run-time limit **will not be marked**. The run-time limits only apply to test cases that are included in this document.

No marks are allocated to commenting in your code. We do however, encourage efficient and well-commented code.

The total marks for the assignment add up to 100, and it will be worth 15% of your overall subject grade.

Part 1: 35 marks

Part 2: 20 marks

Part 3: 45 marks


## Submitting

Before you turn this assignment in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and student ID at the top of this notebook.


Your completed notebook file containing all your answers must be turned in via LMS in `.ipynb` format.
You must also submit a copy of this notebook in `html` format with the output cleared.
You can do this by using the `clear all output` option in the menu.

Your submission should include **only two** files with names formatted as: **Assignment_2.ipynb** and **Assignment_2.html**


## Overview

In this assignment, you will answer questions about VCF files and BLAST.

You will use the `pysam` and `biopython` libraries in your functions. You may want to refer to sections of the documentation for these tools for additional help. Additional to `pysam`, `biopython` and standard Python 3 functions and methods, you may also use any other library we have used in Computational Genomics including `collections`, `numpy`, `pandas`, `math`, `itertools`, `seaborn` and `matplotlib`.

You will also use the `BLAST` command-line tool. 

## Setup

In [5]:
import os
import requests
from IPython.core.display import HTML

# Function to get data. DO NOT MODIFY!
def fetch_file(url, outpath='.'):
    response = requests.get(url)
    if response.status_code == 200:
        print('File found!')
        # Get the filename from the URL
        filename = os.path.basename(url).split('?', 1)[0]
        # Construct the filepath using the specified directory and filename
        filepath = os.path.join(outpath, filename)
        # Create the directory if it doesn't exist
        if not os.path.exists(outpath):
            print(f'Creating output dir: {outpath}')
            os.makedirs(outpath)
        # Check if the file already exists in the specified directory
        if os.path.exists(filepath):
            print(f'{filename} already exists in {outpath}. Skip download.')
        else:
            with open(filepath, 'wb') as f:
                f.write(response.content)
                f.close()
            print(f'Saved to: {filepath}')
    else:
        print(f'File not found: Code {response.status_code}')

/Users/tiknokeziah/Desktop/Keziah/School/COMP90016/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [6]:
# Make the notebook pretty
HTML(requests.get('https://raw.githubusercontent.com/melbournebioinformatics/COMP90016/main/data/2023/style/custom.css').text)

In [7]:
# Fetch assignment data
f_names = ["comp90016_assignment_2_demo_a.vcf",
           "comp90016_assignment_2_demo_b.vcf",
           "comp90016_assignment_2.vcf",
           "comp90016_assignment_2_mbp.vcf",
           "comp90016_assignment_2_COVID19_N.fasta",
           "comp90016_assignment_2_9b.fasta"]

for filename in f_names:
    url = f'https://github.com/melbournebioinformatics/COMP90016/blob/main/data/2024/Assignment_02/data/{filename}?raw=true'
    fetch_file(url,'data')

File found!
comp90016_assignment_2_demo_a.vcf already exists in data. Skip download.
File found!
comp90016_assignment_2_demo_b.vcf already exists in data. Skip download.
File found!
comp90016_assignment_2.vcf already exists in data. Skip download.
File found!
comp90016_assignment_2_mbp.vcf already exists in data. Skip download.
File found!
comp90016_assignment_2_COVID19_N.fasta already exists in data. Skip download.
File found!
comp90016_assignment_2_9b.fasta already exists in data. Skip download.


## Part 1: VCF files

### Setup

In [8]:
# Import the pysam module.
import pysam

In [9]:
# Read in the VCF files to produce VariantFile objects.
demo_vcf_a = pysam.VariantFile("data/comp90016_assignment_2_demo_a.vcf")
demo_vcf_b = pysam.VariantFile("data/comp90016_assignment_2_demo_b.vcf")
comp90016_vcf = pysam.VariantFile("data/comp90016_assignment_2.vcf")

#Confirm object type
comp90016_vcf.is_vcf

True

VariantFile objects allow us to access the data within VCF files. You may want to refer back to the work you did in workshop 5. You may also want to read the documentation linked below:
>https://pysam.readthedocs.io/en/latest/usage.html#working-with-vcf-bcf-formatted-files

### Questions
In the cells below, complete the following tasks:

<div class="question">
<h3>Question 1.1</h3>
    
(5 marks, max 50 words)

Which reference genome (full species name and version) was used for the variant calling  in `comp90016_assignment_2.vcf`? Which variant calling tool was used?

</div>


#### -- GRADED CELL (5 marks) - complete this cell -- 

Full species = Homo sapiens, version = hg38

Variant calling tool = freeBayes v1.3.1

<div class="info">
<h3> Question 1.2 </h3>

(10 marks)  
    
The four standard DNA bases can be divided into purines (A, G) and pyrimidines (C,T). A transition is a single base change from a purine to a purine, or from a pyrimidine to a pyrimidine. A transversion is a single base change from a purine to a pyrimidine or from a pyrimidine to a purine. The transition/transversion ratio (ti/tv) is the ratio of the number of transitions to the number of transversions.
 
<b>Challenge:</b> Write a Python function to calculate the ti/tv for a set of SNVs in a VCF file containing variants from a single sample.

- [ ] Input: a pysam VariantFile object.
- [ ] Output: Return a single floating-point number. Round to 2 decimal places.
- [ ] Disregard variants that are not SNVs.
- [ ] Consider multi-allelic SNVs as seperate SNVs.
- [ ] Consider a particular SNV allele only once, regardless of how many copies of the allele are present.
- [ ] If there are no transversion SNVs, return None.

</div>

In [ ]:
# GRADED CELL 1.2 (10 marks, max 1 min run-time)
%time
def titv(vcf):
    """
    Write a Python function to calculate the ti/tv for a set of SNVs in a VCF file containing variants from a single sample.
    Input: a pysam VariantFile object.
    Output: Return a single floating-point number.
    Disregard variants that are not SNVs.
    Consider multi-allelic SNVs as seperate SNVs.
    Consider a particular SNV allele only once, regardless of how many copies of the allele are present.
    If there are no transversion SNVs, return None.
    """
    
    # YOUR CODE HERE
    type_dict = {"A": "purines", "G": "purines", "C": "pyrimidines", "T": "pyrimidines"}
    snv_dict = {}
    
    ti=0
    tv=0
    
    for variant in vcf.fetch():
        ref = variant.ref
        alt = variant.alts[0]
        
        if len(ref) == 1 and len(alt)== 1:
            snv_dict[variant.pos] = {}
            

            if ref not in snv_dict[variant.pos]:
                snv_dict[variant.pos][ref] = alt
                
                if type_dict[ref] == type_dict[alt]:
                    ti+=1
                elif type_dict[ref] != type_dict[alt]:
                    tv+=1

    if tv != 0:
        ratio = ti/tv
    else:
        ratio = None
    
    return round(ratio, 2)

In [25]:
# Test your function in this cell
print(titv(demo_vcf_a)) # should output 2.0
print(titv(demo_vcf_b)) # should output 1.0

print(titv(comp90016_vcf))

A
('G',)
G
True
0
0
A
('C',)
C
True
0
0
C
('T',)
T
True
0
0
None
A
('<CN0>',)
<CN0>
True
0
0
T
('G',)
G
True
0
0
A
('G',)
G
True
0
0
None
GGACAGACAGACAGACAG
('GGACAGACAGACAG',)
GGACAGACAGACAG
0
0
A
('G',)
G
True
0
0
A
('C',)
C
True
0
0
C
('T',)
T
True
0
0
A
('C',)
C
True
0
0
A
('C',)
C
True
0
0
A
('C',)
C
True
0
0
CGCACGTCCT
('CGCCCGCCCC', 'CGCACGCCCC')
CGCCCGCCCC
0
0
CGCACGCCCC
0
0
A
('G',)
G
True
0
0
G
('C',)
C
True
0
0
T
('C',)
C
True
0
0
C
('T',)
T
True
0
0
G
('A',)
A
True
0
0
CAGTGGGT
('GAGGGGGG', 'CAGGGGGG', 'CAGTGGGG')
GAGGGGGG
0
0
CAGGGGGG
0
0
CAGTGGGG
0
0
T
('C',)
C
True
0
0
C
('T',)
T
True
0
0
A
('G',)
G
True
0
0
G
('A',)
A
True
0
0
T
('C',)
C
True
0
0
T
('C',)
C
True
0
0
A
('G',)
G
True
0
0
T
('C',)
C
True
0
0
A
('G',)
G
True
0
0
A
('G',)
G
True
0
0
A
('G',)
G
True
0
0
TCCA
('CCCC', 'TCCC')
CCCC
0
0
TCCC
0
0
G
('C',)
C
True
0
0
G
('T',)
T
True
0
0
C
('T',)
T
True
0
0
C
('T',)
T
True
0
0
T
('G',)
G
True
0
0
T
('G',)
G
True
0
0
A
('G',)
G
True
0
0
A
('G',)
G
True
0
0
T
('G',)


In [ ]:
# --- AUTOGRADING CELL DO NOT EDIT ----


<div class="question">
<h3>Question 1.3</h3>
    
(10 marks, max 100 words)

Explain why the ti/tv ratio is typically higher in protein-coding regions than in non-coding regions.

</div>


#### -- GRADED CELL (5 marks) - complete this cell -- 

ti/tv ratio high means there are more transitions than transversions. Transitions are favoured in evolution due to their minimal impact on the DNA structure and their tendency to mutate less disruptively on protein function, especially when occurring synonymously. Although nonsynonymous transitions exist, its alterations in amino acid physicochemical properties are less deleterious compared to nonsynonymous transversions. Due to these reasons, transitions are strongly preferred in conserved protein-coding regions where preserving functionality is essential. Additionally, non-coding regions, experiencing lower selective pressure, may have fewer constraints against transversions.

<div class="question">
<h3>Question 1.4</h3>
    
(5 marks, max 100 words)

When working on a variant calling project, you might generate files with the extensions vcf.gz or vcf.tbi. Explain the contents of these files and how they relate to .vcf files.

</div>


#### -- GRADED CELL (5 marks) - complete this cell -- 

vcf.gz is a compressed version of a VCF file, which reduces file size and saves storage space. It has the same information as the uncompressed .vcf file.

vcf.tbi is an index file associated with VCF file. The file contains genomic coordinates (chromosome, start and end positions), binning information (organize genomic regions into bins for a fast and efficient data retrieval), index file, reference sequence metadata etc. With these features, it provides efficient access to genomic regions in the VCF file, rapid querying and retrieval of variants within certain genomic intervals.

<div class="question">
<h3>Question 1.5</h3>
    
(5 marks, max 100 words)

Computational genomics often involves working with large data sets. Explain how .gz and .tbi files help make working with large files feasible.

</div>


#### -- GRADED CELL (5 marks) - complete this cell -- 

Large genomic data sets contain thousands to millions of variants which can occupy storage space. Therefore, compressing using .gz reduces the file size 2x-10x while making it more manageable for storage and data transfer. For .tbi file it plays an important role in accessing specific regions within large genomic data. Since, it indexed the genomic regions into bins, which is like a dictionary, the tool just need to look the indexing up and find particular regions within the compressed VCF file. It makes an efficient retrieval of variants without having to decompress and read the entire file sequentially.

## Part 2: More VCF files

### Setup

The data you will be using are publically available as part of the 1000 Genomes Project, which was completed in 2015. It was one of the first large-scale projects to try and capture the level of variation in the human genome. You can read one of the key publiciations describing this dataset here: https://www.nature.com/articles/nature15393. 

Our gene of interest is *MBP* (Myelin Basic Protein, NM_001025081). Variants in *MBP* from three selected individuals (HG02687, HG03897 and HG04118) have been included.

In [ ]:
# Read in the MBP VCF file to produce a VariantFile objects.
comp90016_vcf_MBP = pysam.VariantFile("data/comp90016_assignment_2_mbp.vcf")

### Questions
In the cells below, complete the following tasks:

<div class="info">
<h3> Question 2.1 </h3>

(10 marks)  
 
<b>Challenge:</b> Write a Python function to create a dictionary of for a given variant position across all samples in a pysam VariantFile object.

- [ ] Input:
    - a pysam VariantFile object (vcf)
    - the name of a contig as a string (chrom)
    - an integer describing a position (pos)
- [ ] Output: A dictionary of genotypes with sample names as keys and tuples of the genotypes as values.
- [ ] If the combination of chrom and pos does not exist in vcf, return None.
    
</div>

In [ ]:
# GRADED CELL 2.1 (10 marks, max 1 min run-time)
%time
def genotype_dict(vcf, chrom, pos):
    """
    Create a genotype dictionary for a given variant position in a pysam VariantFile object. 
    The sample names are keys and tuples of the genotypes are the values. 
    Assume vcf is a pysam VariantFile object. 
    Assume chrom is a string. 
    Assume pos is an integer. 
    If the combination of chrom and pos does not exist in vcf, return None. 
    Return the genotype dictionary.
    """
    
    # YOUR CODE HERE
    geno_dict = {}
    
    for variant in vcf.fetch():
        
        if variant.chrom == chrom and variant.pos == pos:
            
            for geno in vcf.header.samples:
                geno_dict[geno] = variant.samples[geno].values()[0]

    if len(geno_dict) == 0:
        return None
        
    return geno_dict

In [ ]:
# Test your function in this cell
print(genotype_dict(demo_vcf_b, '18', 74690979)) # should output {'HG02687': (0, 0), 'HG03897': (0, 1), 'HG04118': (0, 0)}
print(genotype_dict(demo_vcf_b, '2', 1000)) # should output None

print(genotype_dict(comp90016_vcf_MBP, '18', 74694135))

In [ ]:
# --- AUTOGRADING CELL DO NOT EDIT ----


<div class="question">
<h3>Question 2.2</h3>
    
(5 marks, max 100 words)

For each sample, list the total number of homozygous reference, homozygous alternative and heterozygous alleles. Which of the three samples has the most homozygous alternative variants in the MBP gene?
</div>


In [ ]:
from collections import Counter

counts_dict = {}
inter_count = {}

for i in comp90016_vcf_MBP.fetch():

    for j in i.samples:
        if j not in inter_count:
            inter_count[j] = [i.samples[j]['GT']]
            
        else:
            inter_count[j].append(i.samples[j]['GT'])

for key in inter_count:
    counts_dict[key] = dict(Counter(inter_count[key]))

print(counts_dict)


#### -- GRADED CELL (5 marks) - complete this cell -- 

HG02687: 0|0 (homo. ref) = 1317, 0|1 or 1|0 (hetero.) = 68, 1|1 (homo. alt) = 22
HG03897: 0|0 (homo. ref) = 1349, 0|1 or 1|0 (hetero.) = 42, 1|1 (homo. alt) = 16
HG04118: 0|0 (homo. ref) = 1331, 0|1 or 1|0 (hetero.) = 58, 1|1 (homo. alt) = 18

<div class="question">
<h3>Question 2.3</h3>
    
(5 marks, max 100 words)

What information is contained in the ID column of the MBP VCF file? Explain why this information is useful.
    
</div>


#### -- GRADED CELL (5 marks) - complete this cell -- 

It is a unique label to identify a particular SNVs. They serve as crucial variant identifiers for researchers and databases, offering a standardized naming convention for most SNPs. These IDs are particularly valuable as they facilitate variant annotation and linking to databases like dbSNP (started with 'rs'). This linkage enables researchers to gain deeper insights into variants, including their association with traits, population genetic patterns, and implications for clinical interpretation.

## Part 3: BLAST

### Setup

The gene ORF-9b has been detected in multiple isolates of SARS coronavirus as well as in other coronaviruses that infect non-human animals. We will determine whether SARS-CoV-2 (responsible for COVID-19) has an ORF that is homologous to ORF-9b in other coronaviruses. We will be using `BLAST` on the command line for this purpose.

We will be using a sequence from an isolate of SARS-CoV-2 sequenced early in the COVID-19 pandemic, in December 2019 in Wuhan, China. `data/comp90016_assignment_2_COVID19_N.fasta` is a FASTA file of the DNA sequence for the nucleocapsid (N) gene.

`data/comp90016_assignment_2_9b.fasta` contains ORF-9b sequences from a selection of characterised coronaviruses.

We will be using command line BLAST to execute a search.

BLAST can be downloaded [here](https://www.ncbi.nlm.nih.gov/books/NBK279671/) OR installed using conda `conda install -c bioconda blast`

Execute the following code block to complete the search. Note that `%%bash` signifies that the code block contains Linux commands and not Python code.

In [ ]:
# Import Biopython to take advantage of Bio.Align objects for storing and accessing sequence alignments.
from Bio import Align, Seq, SeqRecord
# You may want to load the BLAST results with BlastIO https://biopython.org/docs/1.75/api/Bio.SearchIO.BlastIO.html

In [ ]:
%%bash
# First let's check that BLAST is installed in the current environment and is available on your PATH
which blastn
# If the output is blank or you see an error, BLAST is not properly installed

In [ ]:
%%bash

# Build a custom BLAST database from the ORF-9b sequences.
echo "Prepare blast database"
makeblastdb -dbtype nucl -in data/comp90016_assignment_2_9b.fasta

# You will see some extra files have appeared.

# Execute a BLAST search.
echo "Running blastn"
# The results will be returned in tabulated form and saved to a file.
blastn -query data/comp90016_assignment_2_COVID19_N.fasta -db data/comp90016_assignment_2_9b.fasta -outfmt 7 -out blast_results.tab

In [ ]:
%%bash
# Print contents of blastn output
cat blast_results.tab

### Questions
In the cells below, complete the following tasks:

<div class="question">

<h3> Question 3.1 </h3>

(5 marks, max 100 words)

BLAST can be used to query a database for matches to a sequence of interest. Here we have created a custom database from a selection of coronavirus genomes, but we may also want to search for matches in a public database such as those available via NCBI, or in a custom database shared by another researcher.
    

Explain one factor that is important to consider when selecting an appropriate BLAST database.
    
</div>



#### -- GRADED CELL (5 marks) - complete this cell -- 

The choice depends on the specific research question or objective at hand. It should contain sequences closely related to those we are querying or relevant to the biological context of our analysis. Factors to consider include the completeness of sequences (whole or fragments), level of annotation, available metadata, database size, frequency of updates for comprehensive and up to date information, and accessibility, as some databases may require specific access controls.

<div class="question">

<h3> Question 3.2 </h3>

(10 marks, max 100 words)

Is there evidence from the BLAST results that the SARS-CoV-2 N gene has an internal ORF homologous to ORF-9b sequences in other coronaviruses? Justify your choice. What was the highest scoring hit?
    
</div>



#### -- GRADED CELL (10 marks) - complete this cell -- 

Yes, the results show multiple hits with high percentage of identities and statistically significant alignment scores (bit scores). The low E-values, approaching 0, indicate a strong confidence level in the matches and a minimal likelihood of the observed similarities occurring randomly. These findings strongly suggest that SARS-CoV-2 N gene and ORG-9b sequences are similar in other coronaviruses. The highest scoring hit achieved a score of 366, further proving the similarity. 

<div class="question">

<h3> Question 3.3 </h3>

(5 marks, max 100 words)

Interpret the E value of the top scoring BLAST hit. How confident are you in your BLAST results?
    
</div>



#### -- GRADED CELL (5 marks) - complete this cell -- 

The E-value describes as the number of hits one can expect to see by chance when searching a database of a particular size. It decreases exponentially as the score of the match increases. The lower E-value the more significant the match is. In the context of the highest scoring BLAST, E-value 7.49e-105, it indicates a very high confidence level in the match. If we were to run the same search against a random database of a smiliar size, we would expect to find a match of this quality or better, less than 1x in 10^105 searches.
Very confident.

<div class="question">

<h3> Question 3.4 </h3>

(5 marks, max 50 words)

Modify the previous BLAST command to:

- Work with an input of a protein sequence FASTA file (named `comp90016_assignment_2_COVID19_N_protein.fasta`). 
- Decrease the word-size parameter from the default value to 2. 
- Use the same custom database and output format used previously. 
    
**Note that you are not required to execute this command.**
    
</div>



#### -- GRADED CELL (5 marks) - complete this cell -- 

blastp -query data/comp90016_assignment_2_COVID19_N_protein.fasta -db data/comp90016_assignment_2_9b.fasta -outfmt 7 -out blast_results.tab -word_size 2

<div class="question">

<h3> Question 3.5 </h3>

(10 marks, max 100 words)

What effect does decreasing the word size have on the sensitivity (ability to detect true homologs) and run-time of the search? Explain why this is the case.
    
</div>



#### -- GRADED CELL (10 marks) - complete this cell -- 

A BLAST search begins by identifying exact matches of the word-size between the query sequence and sequences in the database. Using smaller words increase the chances of discovering short regions of exact matches, enhancing the potential to find seed matches, and increasing sensitivity. Hence, increase the number of detected homologous sequences. However, this approach also leads to longer run-time as it involves processing a greater number of potential seed matches and performing more extensions and evaluations, consuming additional computational resources and time.

For the next question, we will be using amino-acid substitution matrices, encoded as 2D dictionaries. The cell below stores the BLOSUM-62 substitution matrix as a dictionary (the default substitution matrix for BLAST). The protein sequences used to test your code will only contain the one-letter codes of the 20 standard amino-acids.

In [ ]:
BLOSUM_62 = {
    '*':{'*':1,'A':-4,'C':-4,'B':-4,'E':-4,'D':-4,'G':-4,'F':-4,'I':-4,'H':-4,'K':-4,'M':-4,'L':-4,'N':-4,'Q':-4,'P':-4,'S':-4,'R':-4,'T':-4,'W':-4,'V':-4,'Y':-4,'X':-4,'Z':-4},
    'A':{'*':-4,'A':4,'C':0,'B':-2,'E':-1,'D':-2,'G':0,'F':-2,'I':-1,'H':-2,'K':-1,'M':-1,'L':-1,'N':-2,'Q':-1,'P':-1,'S':1,'R':-1,'T':0,'W':-3,'V':0,'Y':-2,'X':-1,'Z':-1},
    'C':{'*':-4,'A':0,'C':9,'B':-3,'E':-4,'D':-3,'G':-3,'F':-2,'I':-1,'H':-3,'K':-3,'M':-1,'L':-1,'N':-3,'Q':-3,'P':-3,'S':-1,'R':-3,'T':-1,'W':-2,'V':-1,'Y':-2,'X':-1,'Z':-3},
    'B':{'*':-4,'A':-2,'C':-3,'B':4,'E':1,'D':4,'G':-1,'F':-3,'I':-3,'H':0,'K':0,'M':-3,'L':-4,'N':3,'Q':0,'P':-2,'S':0,'R':-1,'T':-1,'W':-4,'V':-3,'Y':-3,'X':-1,'Z':1},
    'E':{'*':-4,'A':-1,'C':-4,'B':1,'E':5,'D':2,'G':-2,'F':-3,'I':-3,'H':0,'K':1,'M':-2,'L':-3,'N':0,'Q':2,'P':-1,'S':0,'R':0,'T':-1,'W':-3,'V':-2,'Y':-2,'X':-1,'Z':4},
    'D':{'*':-4,'A':-2,'C':-3,'B':4,'E':2,'D':6,'G':-1,'F':-3,'I':-3,'H':-1,'K':-1,'M':-3,'L':-4,'N':1,'Q':0,'P':-1,'S':0,'R':-2,'T':-1,'W':-4,'V':-3,'Y':-3,'X':-1,'Z':1},
    'G':{'*':-4,'A':0,'C':-3,'B':-1,'E':-2,'D':-1,'G':6,'F':-3,'I':-4,'H':-2,'K':-2,'M':-3,'L':-4,'N':0,'Q':-2,'P':-2,'S':0,'R':-2,'T':-2,'W':-2,'V':-3,'Y':-3,'X':-1,'Z':-2},
    'F':{'*':-4,'A':-2,'C':-2,'B':-3,'E':-3,'D':-3,'G':-3,'F':6,'I':0,'H':-1,'K':-3,'M':0,'L':0,'N':-3,'Q':-3,'P':-4,'S':-2,'R':-3,'T':-2,'W':1,'V':-1,'Y':3,'X':-1,'Z':-3},
    'I':{'*':-4,'A':-1,'C':-1,'B':-3,'E':-3,'D':-3,'G':-4,'F':0,'I':4,'H':-3,'K':-3,'M':1,'L':2,'N':-3,'Q':-3,'P':-3,'S':-2,'R':-3,'T':-1,'W':-3,'V':3,'Y':-1,'X':-1,'Z':-3},
    'H':{'*':-4,'A':-2,'C':-3,'B':0,'E':0,'D':-1,'G':-2,'F':-1,'I':-3,'H':8,'K':-1,'M':-2,'L':-3,'N':1,'Q':0,'P':-2,'S':-1,'R':0,'T':-2,'W':-2,'V':-3,'Y':2,'X':-1,'Z':0},
    'K':{'*':-4,'A':-1,'C':-3,'B':0,'E':1,'D':-1,'G':-2,'F':-3,'I':-3,'H':-1,'K':5,'M':-1,'L':-2,'N':0,'Q':1,'P':-1,'S':0,'R':2,'T':-1,'W':-3,'V':-2,'Y':-2,'X':-1,'Z':1},
    'M':{'*':-4,'A':-1,'C':-1,'B':-3,'E':-2,'D':-3,'G':-3,'F':0,'I':1,'H':-2,'K':-1,'M':5,'L':2,'N':-2,'Q':0,'P':-2,'S':-1,'R':-1,'T':-1,'W':-1,'V':1,'Y':-1,'X':-1,'Z':-1},
    'L':{'*':-4,'A':-1,'C':-1,'B':-4,'E':-3,'D':-4,'G':-4,'F':0,'I':2,'H':-3,'K':-2,'M':2,'L':4,'N':-3,'Q':-2,'P':-3,'S':-2,'R':-2,'T':-1,'W':-2,'V':1,'Y':-1,'X':-1,'Z':-3},
    'N':{'*':-4,'A':-2,'C':-3,'B':3,'E':0,'D':1,'G':0,'F':-3,'I':-3,'H':1,'K':0,'M':-2,'L':-3,'N':6,'Q':0,'P':-2,'S':1,'R':0,'T':0,'W':-4,'V':-3,'Y':-2,'X':-1,'Z':0},
    'Q':{'*':-4,'A':-1,'C':-3,'B':0,'E':2,'D':0,'G':-2,'F':-3,'I':-3,'H':0,'K':1,'M':0,'L':-2,'N':0,'Q':5,'P':-1,'S':0,'R':1,'T':-1,'W':-2,'V':-2,'Y':-1,'X':-1,'Z':3},
    'P':{'*':-4,'A':-1,'C':-3,'B':-2,'E':-1,'D':-1,'G':-2,'F':-4,'I':-3,'H':-2,'K':-1,'M':-2,'L':-3,'N':-2,'Q':-1,'P':7,'S':-1,'R':-2,'T':-1,'W':-4,'V':-2,'Y':-3,'X':-1,'Z':-1},
    'S':{'*':-4,'A':1,'C':-1,'B':0,'E':0,'D':0,'G':0,'F':-2,'I':-2,'H':-1,'K':0,'M':-1,'L':-2,'N':1,'Q':0,'P':-1,'S':4,'R':-1,'T':1,'W':-3,'V':-2,'Y':-2,'X':-1,'Z':0},
    'R':{'*':-4,'A':-1,'C':-3,'B':-1,'E':0,'D':-2,'G':-2,'F':-3,'I':-3,'H':0,'K':2,'M':-1,'L':-2,'N':0,'Q':1,'P':-2,'S':-1,'R':5,'T':-1,'W':-3,'V':-3,'Y':-2,'X':-1,'Z':0},
    'T':{'*':-4,'A':0,'C':-1,'B':-1,'E':-1,'D':-1,'G':-2,'F':-2,'I':-1,'H':-2,'K':-1,'M':-1,'L':-1,'N':0,'Q':-1,'P':-1,'S':1,'R':-1,'T':5,'W':-2,'V':0,'Y':-2,'X':-1,'Z':-1},
    'W':{'*':-4,'A':-3,'C':-2,'B':-4,'E':-3,'D':-4,'G':-2,'F':1,'I':-3,'H':-2,'K':-3,'M':-1,'L':-2,'N':-4,'Q':-2,'P':-4,'S':-3,'R':-3,'T':-2,'W':11,'V':-3,'Y':2,'X':-1,'Z':-3},
    'V':{'*':-4,'A':0,'C':-1,'B':-3,'E':-2,'D':-3,'G':-3,'F':-1,'I':3,'H':-3,'K':-2,'M':1,'L':1,'N':-3,'Q':-2,'P':-2,'S':-2,'R':-3,'T':0,'W':-3,'V':4,'Y':-1,'X':-1,'Z':-2},
    'Y':{'*':-4,'A':-2,'C':-2,'B':-3,'E':-2,'D':-3,'G':-3,'F':3,'I':-1,'H':2,'K':-2,'M':-1,'L':-1,'N':-2,'Q':-1,'P':-3,'S':-2,'R':-2,'T':-2,'W':2,'V':-1,'Y':7,'X':-1,'Z':-2},
    'X':{'*':-4,'A':-1,'C':-1,'B':-1,'E':-1,'D':-1,'G':-1,'F':-1,'I':-1,'H':-1,'K':-1,'M':-1,'L':-1,'N':-1,'Q':-1,'P':-1,'S':-1,'R':-1,'T':-1,'W':-1,'V':-1,'Y':-1,'X':-1,'Z':-1},
    'Z':{'*':-4,'A':-1,'C':-3,'B':1,'E':4,'D':1,'G':-2,'F':-3,'I':-3,'H':0,'K':1,'M':-1,'L':-3,'N':0,'Q':3,'P':-1,'S':0,'R':0,'T':-1,'W':-3,'V':-2,'Y':-2,'X':-1,'Z':4}}

<div class="info">
<h3> Question 3.6 </h3>

(10 marks)

<b>Challenge:</b> Write a Python function to calculate the percentage sequence similarity between two aligned protein sequences. The percentage similarity is defined as the number of positions with similar amino acid residues as a percentage of the total number of positions in the alignment. 
    
Two amino acid residues are considered similar based on the input substitution matrix. If two amino acids have a value greater than 0 in their corresponding cell of the input matrix, they are similar. 

- [ ] Input: 
    - Pairwise_aln as a `Bio.Align.MultipleSeqAlignment` object that contains only two aligned sequences.
    - sub_matrix as a 2D dictionary encoding an amino-acid substitution matrix, such as the BLOSUM-62 matrix above.
- [ ] Output: Return a floating-point number between 0 and 100. 
- [ ] Round the output to 4 decimal places.
- [ ] If pairwise_aln does not contain 2 sequences, return None.

</div>

In [ ]:
# GRADED CELL 3.6 (10 marks, max 1 min run-time)
%time
def percentage_similarity(pairwise_aln, sub_matrix):
    """
    Calculate the percentage sequence similarity between two aligned protein sequences. 
    Assume that pairwise_aln is an Bio.Align.MultipleSeqAlignment object. 
    Assume sub_matrix is a 2D dictionary encoding an amino-acid substitution matrix. 
    Two amino acid residues are similar if there is a value greater than 0 in the corresponding cell of sub_matrix. 
    The percentage similarity is defined as the number of positions with similar amino acid residues as a percentage of the total number of positions in the alignment.
    Return a floating point number between 0 and 100.
    Round the output to 4 decimal places.
    If pairwise_aln does not contain 2 sequences, return None.
    """
   
    # YOUR CODE HERE
    similar_aa = 0
    
    if len(pairwise_aln) == 2:
        seq1 = pairwise_aln[0].seq
        seq2 = pairwise_aln[1].seq

        for s1,s2 in zip(seq1,seq2):
            if s1 != '-' and s2 != '-' and sub_matrix[s1][s2] > 0:
                similar_aa += 1
            else:
                continue

        similarity = similar_aa / len(seq1) * 100
        
    else:
        return None

    return round(similarity, 4)

In [ ]:
# ~~ Test your function in this cell ~~
demo_sequence_a = SeqRecord.SeqRecord(Seq.Seq("MYWIW"))
demo_sequence_b = SeqRecord.SeqRecord(Seq.Seq("IYW--"))
demo_pairwise_prot_aln = Align.MultipleSeqAlignment([demo_sequence_a, demo_sequence_b])

sequence_a = SeqRecord.SeqRecord(Seq.Seq("MYGEGEPGGWQDHVTVLATRRHPKWAQAWVSTMPWGYECGFSRAWVHQTPWINV-----VSLSSHEAYGVVAVRHPWEIFSPYEVYAPYVQDTQHHGNPGQFTTSCYPDE"))
sequence_b = SeqRecord.SeqRecord(Seq.Seq("MYADGEPGAWQDHMTVLAIYWHHKWAHAWVSTMPWSYECGFSRAWVHQTPWINVIRFTQVSLSSRAWYGILAVRHPWEIFSPYDVYAPYVAATQHHGNPGQFSTSCYP--"))
pairwise_prot_aln = Align.MultipleSeqAlignment([sequence_a, sequence_b])

print(percentage_similarity(demo_pairwise_prot_aln, BLOSUM_62)) # should return 60.0

print(percentage_similarity(pairwise_prot_aln, BLOSUM_62))

In [ ]:
# --- AUTOGRADING CELL DO NOT EDIT ----


# END OF ASSIGNMENT

## Submitting

Follow these steps to submit your assignment

1) Before you turn this assignment in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

2) Make sure you have filled in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE"

3) **Clear all output** (in the menubar, select Kernel$\rightarrow$Restart & Clear Output)

4) Your completed notebook file containing all your answers must be turned in via Canvas in `.ipynb` format.

5) You must also submit a copy of this notebook in `html` format with the output cleared.


Your submission should include **only two** files with names formatted as: **Assignment_2.ipynb** and **Assignment_2.html**
